In [1]:
import numpy as np
import pymol as cmd
from pymol import cmd,stored

#Function 1 extracting the residues around cystein (CYS, CSD, CSO, OCS) 
#within a cutoff (dist_cutoff), in a protein (protname)

def Cys_environment(cysname, protname, dist_cutoff):

    l= dist_cutoff #distance to the cystein
    
    ###. fetching the Protein form the PDB in their biological essembly type="pdb1"
    cmd.fetch(protname, "Prot", type="pdb1")
    cmd.split_states("Prot")
    cmd.remove("Prot")
    Obj=cmd.get_object_list()
    
    ### Selecting the Cystein and identifying their index number 
    list_results=[]
    y=Obj[0]
    CysSelect=(f"{y} and resname {cysname}") #### Selection for residue
    cmd.select('Cys', CysSelect)
    stored.list=[]
    cmd.iterate("(Cys & name ca)", "stored.list.append((resi,resn))");

    
    #### Neigbors to Cysteines by residue index
    if len(stored.list)==0:
        var=[]
        stored.amino=[]
        list_results=[]
        x=[]
        i=[]
    else:

        i=0
        while i<len(stored.list):
            x=stored.list[i][0]
            var =(f"br. all nto. {l} of ({y} and resi {x})")
            cmd.select('Res', var)
            stored.amino=[]
            cmd.iterate("(Res and name o)","stored.amino.append((resn))");
            list_results.append(stored.amino)
            i=i+1
    list_results2=[*list_results]
    cmd.remove('all')
    del l, var, x, i, list_results, CysSelect, stored.list, stored.amino, Obj
    return list_results2

#### Function 2 extract all the information from a list of proteins (pdbid) in AAList. 

def extract_env (AAList, Cys_name, dist_cutoff):
    L=[]
    Lcount=0
    for i in AAList:
        print(i)
        L1=Cys_environment(Cys_name, i, dist_cutoff)
        Lcount=Lcount + len(L1)
        flat_list = [item for sublist in L1 for item in sublist]
        L.append(*[flat_list])
        del flat_list, L1
    LOut=[item for sublist in L for item in sublist]
    return Lcount, LOut

In [2]:
### writing out the list of neigboring residues in a *txt file from
### a list of proteins (list_pdb) chaange the Cysteine name to either CYS, CSD, CSO, OCS

AAListALL = open('list_pdb2', 'r').read().splitlines()
AAListInput=np.unique(AAListALL)
dist_cutoff=3.6

extract_env(AAListInput, 'CSD', dist_cutoff)

### extrating enviroment for CSD
text_file = open("CSD_enviroment2.txt", "w+")
text_file.write(np.str(extract_env(AAListInput, 'CSD', dist_cutoff)))
text_file.close()
